# <div align="center"> k12学习框架错误码 </div>

## 一. 统一错误码

|错误码|简述|中文描述 | 其他 |
|:---:|:------|:------|:-----|
| 100000 | success | 通用操作成功 | |
| 100100 | 100100 | 预留, 表示api传过来的参数是对的 | |
| 100101 | api parameter key is not found | api传递参数中缺少关键key或者key不正确 | api |
| 100102 | api parameter value is invalid | api传递参数key对应的值错误或者和约定不一致 | api |
| 100199 | api parameter unkown exception | api内部不可知的错误 | api | 
| 100200 | task service success | 每个框架及平台接口都是以服务提供功能, 表示这些服务工作正常 | |
| 100201 | task service is not found | 用户请求的任务相关的服务不存在或者该服务没有启动 | service |
| 100202 | task service start fails | 用户请求的任务启动失败 | service |
| 100299 | task service unkown exception | 任务服务相关的不可知的错误 | service |
| 100300 | k12cv container success | 每个cv框架任务对应一个container进程, 100300表示进程启动成功 | |
| 100301 | k12cv container is not found! | (cv)指定的container不存在 | k12cv |
| 100302 | k12cv container start fail | (cv)container启动失败 | k12cv |
| 100303 | k12cv container stop fail | (cv)停止指定的container失败 | k12cv |
| 100304 | k12cv container is already running | (cv)指定的container已经启动 | k12cv |
| 100399 | k12cv container unkown exception | (cv)框架上不可知的异常 | k12cv |
| 100400 | k12nlp container success | 每个nlp框架任务对应一个container进程, 100400表示进程启动成功 | |
| 100401 | k12nlp container is not found | (nlp)指定的container不存在 | k12nlp |
| 100402 | k12nlp container start fail | (nlp)container启动失败| k12nlp |
| 100403 | k12nlp container stop fail | (nlp)停止指定的container失败 | k12nlp |
| 100404 | k12nlp container is already running | (nlp)指定的container已经启动 | k12nlp |
| 100405 | k12nlp container process start configure error | (nlp)框架上启动的进程解析配置文件失败 | k12nlp |
| 100499 | k12nlp continaer process unkown exception | (nlp)框架上不可知的异常 | k12nlp |
| 100500 | 100500 | | 预留 |
| 100600 | 100600 | | 预留 | 
| 100700 | 100700 | | 预留 |
| 100800 | 100800 | | 预留 |
| 100900 | 100900 | 通用的异常 | | 
| 100901 | common exception: out of memory | 内存不足 | |
| 999999 | Unkown error! | 不可知的错误 | |


[最新错误码定义](https://gitee.com/hzcsai_com/hzcsk12/blob/master/k12ai_errmsg.py)

## 二. 错误码模型定义

```json
{
    "code": 101,  // [M] 统一错误码
    "descr": "",  // [M] 错误简要信息
    "message":"", // [O] 自定义消息,辅助消息
    "detail": {     // [O] 错误详细信息
        "filename": "", // [C] 非异常错误时, 错误输出所在文件
        "linenum": 100, // [C] 非异常错误时, 错误输出所在行号
        "funcname": "", // [C] 非异常错误时, 错误输出所在函数
        "err_type": "", // [C] 错误是异常时, 异常的类名
        "err_text": "", // [C] 错误是异常时, 异常的内容
        "traceback":[   // [C] 错误是异常时, 错误堆栈
            {
                "filename": "",
                "linenum": 100,
                "funcname": "",
                "source": "" // [C] 错误是异常时, 发生问题的程序代码
            },
            {
                "filename": "",
                "linenum": 100,
                "funcname": "",
                "source": "" // [C] 错误是异常时, 发生问题的程序代码
            },
        ]
    }
}
```

## 四. 案例

In [1]:
import requests
import json
import consul
import time

hostname = 'gamma'
host = '10.255.20.227'
port = 8129

user = 'test'
uuid = '123456'

consul_addr = host
consul_port = 8500

In [2]:
# API异步接口将过程中产出的数据同步到consul数据中心, 调用该接口可从中心获取数据 (数据中心仅供开发使用)
def query_async_data(key, wait = 1):
    time.sleep(wait)
    try:
        client = consul.Consul(consul_addr, consul_port)
        _, data = client.kv.get(key)
        return str(data['Value'], encoding="utf-8")
    except:
        return 'Query nothing, try again later!'
    
# 格式化打印json字符串
def print_json(text):
    if isinstance(text, str):
        print(json.dumps(json.loads(text), indent=4))
    else:
        print(json.dumps(text, indent=4))

In [3]:
config = '''{
    "dataset_reader":{
        "type": "sst_tokens",
        "use_subtrees": true,
        "granularity": "2-class"
    },
    "validation_dataset_reader":{
        "type": "sst_tokens",
        "use_subtrees": false,
        "granularity": "2-class"
    },
    "train_data_path": "/data/datasets/nlp/sst/train.txt",
    "validation_data_path": "/data/datasets/nlp/sst/dev.txt",
    "test_data_path": "/data/datasets/nlp/sst/test.txt",  
    "model": {
        "type": "basic_classifier",        
        "text_field_embedder": {
            "token_embedders": {
                "tokens": {
                    "type": "embedding",
                    "embedding_dim": 300,          
                    "trainable": false          
                }
            }
        },
        "seq2vec_encoder": {
            "type": "lstm",                     
            "input_size": 300,
            "hidden_size": 512,
            "num_layers": 2,
            "batch_first": true
        }
    },    
    "iterator": {
        "type": "bucket",
        "sorting_keys": [["tokens", "num_tokens"]],
        "batch_size" : 32
    },
    "trainer": {
        "num_epochs": 5,
        "patience": 1,
        "grad_norm": 5.0,
        "validation_metric": "+accuracy",
        "cuda_device": 0,
        "optimizer": {
            "type": "adam",
            "lr": 0.001
        }
    }
}'''

### 1. 缺少参数

In [4]:
data = '''{
    "xxx":"train.start",
    "user": "%s",
    "service_name": "k12nlp",
    "service_uuid": "%s",
    "service_params": %s
}''' % (user, uuid, config)

api = 'http://%s:%d/k12ai/framework/train' % (host, port)
print_json(requests.post(url=api, json=json.loads(data)).text)

{
    "code": 100101,
    "descr": "api parameter key is not found",
    "detail": {
        "exc_type": "KeyError",
        "exc_text": "'op'",
        "trackback": [
            {
                "filename": "k12ai_service.py",
                "linenum": 164,
                "funcname": "_framework_train",
                "souce": "op = reqjson['op']"
            }
        ]
    }
}


### 2. 参数值错误

In [5]:
data = '''{
    "op":"xxx",
    "user": "%s",
    "service_name": "k12nlp",
    "service_uuid": "%s",
    "service_params": %s
}''' % (user, uuid, config)

api = 'http://%s:%d/k12ai/framework/train' % (host, port)
print_json(requests.post(url=api, json=json.loads(data)).text)

{
    "code": 100102,
    "descr": "api parameter value is invalid",
    "message": "not support op:xxx"
}


### 3. 服务不存在

In [6]:
data = '''{
    "op":"train.start",
    "user": "%s",
    "service_name": "xxx",
    "service_uuid": "%s",
    "service_params": %s
}''' % (user, uuid, config)

api = 'http://%s:%d/k12ai/framework/train' % (host, port)
print_json(requests.post(url=api, json=json.loads(data)).text)

{
    "code": 100201,
    "descr": "task service is not found",
    "message": "service name:xxx"
}


### 4. 服务启动失败

In [7]:
config_err = config.replace('lstm', 'xxx')
data = '''{
    "op":"train.start",
    "user": "%s",
    "service_name": "k12nlp",
    "service_uuid": "%s",
    "service_params": %s
}''' % (user, uuid, config_err)

api = 'http://%s:%d/k12ai/framework/train' % (host, port)
print_json(requests.post(url=api, json=json.loads(data)).text)

{
    "code": 100200,
    "descr": "task service success",
    "message": "train.start task cache directory: /data/users/test/123456"
}


In [9]:
# sleep 10s
print(query_async_data('framework/%s/%s/train.start/error'%(user, uuid)))

{
    "version": "0.1.0",
    "type": "error",
    "tag": "framework",
    "op": "train.start",
    "user": "test",
    "service_uuid": "123456",
    "timestamp": 1574419278656,
    "datetime": "2019-11-22 18:41:18",
    "error": {
        "code": 100405,
        "descr": "k12nlp container process start configure error",
        "detail": {
            "exc_type": "ConfigurationError",
            "exc_text": "'xxx not in acceptable choices for model.seq2vec_encoder.type: [\\'cnn\\', \\'cnn-highway\\', \\'bert_pooler\\', \\'bag_of_embeddings\\', \\'boe\\', \\'gru\\', \\'lstm\\', \\'rnn\\', \\'augmented_lstm\\', \\'alternating_lstm\\', \\'stacked_bidirectional_lstm\\']. You should either use the --include-package flag to make sure the correct module is loaded, or use a fully qualified class name in your config file like {\"model\": \"my_module.models.MyModel\"} to have it imported automatically.'",
            "trackback": [
                {
                    "filename": "/hzcsk12/al